# Baseline Model Architecture

> Defines the baseline LLaVA 1.5 model architecture, including the projector and the combined model structure.

In [ ]:
#| default_exp model.baseline

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import sys
from pathlib import Path
import os

# Assumes the notebook is run from the project root or one level down (e.g., nbs/)
# Navigate up to the project root (where settings.ini or .git likely exists)
project_root = Path(os.getcwd())
# Simple check: If settings.ini is not in cwd, assume we are in nbs/ and go up one level
if not (project_root / 'settings.ini').exists() and (project_root.parent / 'settings.ini').exists():
    project_root = project_root.parent

project_root_str = str(project_root.resolve())

if project_root_str not in sys.path:
    print(f"Adding project root to sys.path: {project_root_str}")
    sys.path.insert(0, project_root_str)
else:
    print(f"Project root already in sys.path: {project_root_str}")

Adding project root to sys.path: /workspace/llava


In [ ]:
#| export
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence # For padding variable length sequences
from transformers import CLIPVisionModel, AutoModelForCausalLM, AutoConfig
from transformers.modeling_outputs import BaseModelOutputWithPooling, CausalLMOutputWithPast # For type hints
import warnings
from typing import Optional # Added for type hinting

try:
    from peft import LoraConfig, get_peft_model, PeftModel # Import PEFT components
    _peft_available = True
except ImportError:
    print("Warning: peft library not found. LoRA functionality will be disabled.")
    LoraConfig, get_peft_model, PeftModel = None, None, None # Define as None if not available
    _peft_available = False

from llava.utils import load_config # Assuming utils notebook is created
from llava.data.preprocessing import IMAGE_TOKEN_INDEX_PLACEHOLDER, IGNORE_INDEX, tokenizer, DEFAULT_IMAGE_TOKEN # Import constants and tokenizer

# Ensure tokenizer is loaded (it should be from the import)
if tokenizer is None:
    print("Warning: Tokenizer could not be imported from data.preprocessing. Trying to load it again...")
    try:
        # Need to load config first to get LLM name
        _config_temp = load_config('configs/config.yaml')
        _llm_name_temp = _config_temp.get('model', {}).get('llm_name_or_path', 'lmsys/vicuna-7b-v1.5')
        _tokenizer_max_len_temp = _config_temp.get('data', {}).get('tokenizer_model_max_length', 2048)
        _tokenizer_padding_side_temp = _config_temp.get('data', {}).get('tokenizer_padding_side', 'right')

        from transformers import AutoTokenizer
        tokenizer = AutoTokenizer.from_pretrained(
            _llm_name_temp,
            model_max_length=_tokenizer_max_len_temp,
            padding_side=_tokenizer_padding_side_temp,
            use_fast=True,
        )
        # Re-add special token logic if necessary (should match preprocessing step)
        # from llava.data.preprocessing import DEFAULT_IMAGE_TOKEN # Already imported
        if DEFAULT_IMAGE_TOKEN not in tokenizer.get_vocab():
             num_added = tokenizer.add_special_tokens({'additional_special_tokens': [DEFAULT_IMAGE_TOKEN]})
             print(f"Added {num_added} token(s) during re-load. New vocab size: {len(tokenizer)}")
        if tokenizer.pad_token is None:
             tokenizer.pad_token = tokenizer.eos_token
             print(f"Set pad token to EOS during re-load.")
        print(f"Tokenizer re-loaded successfully for {_llm_name_temp}.")

    except Exception as e:
        print(f"Fatal Error: Could not load tokenizer. {e}")
        # Depending on the context, might raise an error or exit
        raise ImportError("Tokenizer is essential and could not be loaded.") from e

Project root already in sys.path: /workspace/llava
Loaded config from configs/config.yaml
Successfully loaded tokenizer for: lmsys/vicuna-7b-v1.5
Adding special token <image> to tokenizer.
Added 1 token(s). New vocab size: 32001
Using token ID for <image>: 32000
Tokenizer already has pad token: <unk> (ID: 0)


## Step 2.1: Define Projector Module

The projector module connects the vision encoder's output to the language model's input space. In LLaVA 1.5, this is typically a simple MLP.

In [ ]:
#| export
class LLaVAProjector(nn.Module):
    """A simple 2-layer MLP projector to map vision features to LLM embedding space.

    Maps input_dim (e.g., CLIP ViT-L output dim = 1024) to
    output_dim (e.g., Vicuna-7B hidden dim = 4096).

    Architecture: Linear -> GELU -> Linear
    """
    def __init__(self, input_dim: int, output_dim: int):
        """Initializes the MLP projector.

        Args:
            input_dim: Dimension of the input vision features.
            output_dim: Dimension of the LLM's hidden space.
        """
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, output_dim, bias=True),
            nn.GELU(),
            nn.Linear(output_dim, output_dim, bias=True)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Projects the input features.

        Args:
            x: Input tensor of shape (batch_size, num_patches, input_dim) or similar.

        Returns:
            Projected tensor of shape (batch_size, num_patches, output_dim).
        """
        return self.model(x)

In [ ]:
show_doc(LLaVAProjector)

---

### LLaVAProjector

>      LLaVAProjector (input_dim:int, output_dim:int)

*A simple 2-layer MLP projector to map vision features to LLM embedding space.

    Maps input_dim (e.g., CLIP ViT-L output dim = 1024) to
    output_dim (e.g., Vicuna-7B hidden dim = 4096).

    Architecture: Linear -> GELU -> Linear*

In [ ]:
#| test
# Test projector instantiation and forward pass
inp_dim = 1024
out_dim = 4096
projector = LLaVAProjector(input_dim=inp_dim, output_dim=out_dim)

# Create a dummy input tensor
batch_size = 2
num_patches = 576 # Example for 336x336 input with patch size 14
dummy_input = torch.randn(batch_size, num_patches, inp_dim)

# Perform forward pass
output = projector(dummy_input)

# Check output shape
assert output.shape == (batch_size, num_patches, out_dim), f"Expected shape {(batch_size, num_patches, out_dim)}, but got {output.shape}"
print("Projector test passed!")

Projector test passed!


---

## Step 2.2, 2.3, 2.4, 4.2: Define Baseline LLaVA Model & Integrate Components + LoRA

This class combines the Vision Encoder, the Projector, and the Language Model to form the complete LLaVA baseline model. We integrate the loading of the Vision Encoder (CLIP) and the Language Model (Vicuna), handle necessary configurations like embedding resizing, and **now integrate optional LoRA application using the `peft` library.**

In [ ]:
#| export
class BaselineLLaVAModel(nn.Module):
    """Baseline LLaVA 1.5 model combining Vision Encoder, Projector, and LLM.

    Loads the specified Vision Encoder (CLIP) and LLM (Vicuna) from Hugging Face Hub,
    along with the Projector. Handles freezing components, LLM embedding resizing,
    and optionally applies LoRA to the LLM based on the configuration.
    Implements `encode_image`, activation checkpointing, and the main `forward` pass.
    """
    def __init__(self, config: dict):
        """Initializes the Baseline LLaVA model structure.

        Loads Vision Encoder, LLM, and Projector based on the config.
        Resizes LLM embeddings to match the tokenizer vocabulary size.
        Optionally applies LoRA to the LLM.
        Optionally applies activation checkpointing to the LLM.

        Args:
            config: Dictionary containing model configuration, including paths/names
                    for vision encoder, LLM, projector dimensions, LoRA settings, checkpointing, etc.
        """
        super().__init__()
        self.config = config # Store the main config
        self.model_config = config.get('model', {}) # Store the model sub-config

        # Initialize components placeholders
        self.vision_tower = None
        self.language_model = None
        self.projector = None

        # Store configuration details needed
        self.image_token_index_marker = self.model_config.get('image_token_index_marker', IMAGE_TOKEN_INDEX_PLACEHOLDER)
        self.ignore_index = IGNORE_INDEX # Use imported constant
        self.vision_feature_layer = self.model_config.get('vision_feature_layer', -2)
        self.vision_encoder_name = self.model_config.get('vision_encoder_name_or_path', 'openai/clip-vit-large-patch14-336')
        self.llm_name = self.model_config.get('llm_name_or_path', 'lmsys/vicuna-7b-v1.5')

        # --- Load component configs first --- #
        llm_hf_config = AutoConfig.from_pretrained(self.llm_name)
        vision_hf_config = AutoConfig.from_pretrained(self.vision_encoder_name)

        # --- Determine Projector dimensions --- #
        projector_cfg_from_yaml = self.model_config.get('projector', {})
        # Input: Use vision model's hidden size
        proj_input_dim = projector_cfg_from_yaml.get('input_dim', vision_hf_config.hidden_size)
        # Output: Use LLM's hidden size
        proj_output_dim = projector_cfg_from_yaml.get('output_dim', llm_hf_config.hidden_size)

        # --- Initialize Projector --- #
        print(f"Initializing Projector: Input Dim={proj_input_dim}, Output Dim={proj_output_dim}")
        self.projector = LLaVAProjector(proj_input_dim, proj_output_dim)

        # --- Load Vision Tower --- #
        self.load_vision_tower(expected_output_dim=proj_input_dim)

        # --- Load Language Model (and potentially apply LoRA) --- #
        self.load_language_model()

        # --- Resize LLM Embeddings --- #
        self.resize_llm_embeddings()
        
        # --- Apply Activation Checkpointing if enabled --- #
        self.apply_activation_checkpointing()

    def load_vision_tower(self, expected_output_dim: int):
        """Loads the CLIP Vision Model based on the configuration.
        Args:
            expected_output_dim: The dimension the projector expects as input,
                                 used for validation against the loaded vision tower.
        """
        try:
            print(f"Loading Vision Tower: {self.vision_encoder_name}...")
            self.vision_tower = CLIPVisionModel.from_pretrained(self.vision_encoder_name)
            print(f"Vision Tower loaded successfully.")

            # Freeze the vision tower by default (LLaVA Stage 1 & commonly Stage 2)
            self.vision_tower.requires_grad_(False)
            print(f"Vision Tower weights frozen.")

            # Check if vision tower output dim matches projector input dim
            vision_output_dim = self.vision_tower.config.hidden_size
            if vision_output_dim != expected_output_dim:
                warnings.warn(
                    f"Vision Tower output dimension ({vision_output_dim}) does not match "
                    f"Projector input dimension ({expected_output_dim}). Check vision model and config['model']['projector']['input_dim']."
                )
        except Exception as e:
            print(f"Error loading Vision Tower ({self.vision_encoder_name}): {e}")
            print("Please ensure the model name is correct and you have internet connectivity.")
            self.vision_tower = None # Set to None if loading fails

    def load_language_model(self):
        """Loads the Language Model and optionally applies LoRA based on the configuration."""
        try:
            print(f"Loading Language Model: {self.llm_name}...")
            # TODO: Add logic for 4-bit loading (QLoRA) here if needed, requires bitsandbytes
            # Example: load_in_4bit=self.model_config.get('quantization', {}).get('load_in_4bit', False)
            self.language_model = AutoModelForCausalLM.from_pretrained(self.llm_name)
            print(f"Language Model loaded successfully.")

            # Freeze the base LLM weights before applying PEFT
            self.language_model.requires_grad_(False)
            print(f"Base Language Model weights frozen.")

            # --- Apply PEFT/LoRA (Step 4.2) --- 
            peft_config = self.model_config.get('peft', {})
            use_lora = peft_config.get('use_lora', False)

            if use_lora:
                if not _peft_available:
                    print("Warning: `use_lora` is true in config, but peft library is not installed. Skipping LoRA.")
                else:
                    print("Applying LoRA...")
                    lora_r = peft_config.get('lora_r', 8)
                    lora_alpha = peft_config.get('lora_alpha', 16)
                    lora_dropout = peft_config.get('lora_dropout', 0.05)
                    target_modules = peft_config.get('target_modules', ['q_proj', 'v_proj']) # Default targets
                    # TODO: Add task_type (CAUSAL_LM) if needed by peft version
                    # from peft import TaskType
                    
                    lora_config = LoraConfig(
                        r=lora_r,
                        lora_alpha=lora_alpha,
                        target_modules=target_modules,
                        lora_dropout=lora_dropout,
                        bias="none", # Common setting for LoRA
                        # task_type=TaskType.CAUSAL_LM # Specify task type if needed
                    )
                    
                    self.language_model = get_peft_model(self.language_model, lora_config)
                    print(f"LoRA applied. Config: r={lora_r}, alpha={lora_alpha}, dropout={lora_dropout}, modules={target_modules}")
                    # Print trainable parameters after applying LoRA
                    self.language_model.print_trainable_parameters()
            else:
                print("LoRA is disabled in the configuration.")
            # ----------------------------------

        except Exception as e:
            print(f"Error loading Language Model ({self.llm_name}): {e}")
            print("Please ensure the model name is correct and you have internet connectivity / memory.")
            self.language_model = None # Set to None if loading fails

    def resize_llm_embeddings(self):
        """Resizes the LLM's token embeddings to match the tokenizer size."""
        if self.language_model is None or tokenizer is None:
            print("Warning: Cannot resize LLM embeddings. LLM or tokenizer not available.")
            return

        current_embeddings = self.get_input_embeddings() # Use helper to handle PEFT
        current_vocab_size = current_embeddings.weight.size(0)
        target_vocab_size = len(tokenizer)

        if current_vocab_size != target_vocab_size:
            print(f"Resizing LLM token embeddings from {current_vocab_size} to {target_vocab_size} (tokenizer size)...")
            self.language_model.resize_token_embeddings(target_vocab_size)
            # Initialization of new embeddings is handled by resize_token_embeddings
            print("LLM token embeddings resized.")
        else:
            print("LLM embedding size already matches tokenizer size. No resizing needed.")
            
    def apply_activation_checkpointing(self):
        """Applies gradient checkpointing if enabled in the config."""
        use_checkpointing = self.model_config.get('use_activation_checkpointing', False)
        if use_checkpointing:
            if self.language_model is not None:
                 # Check if it's a PeftModel, need to call on base_model if so
                 # The `gradient_checkpointing_enable` method exists on the PeftModel wrapper too
                 model_to_checkpoint = self.language_model 
                 
                 # Check if the model has the gradient_checkpointing_enable method
                 if hasattr(model_to_checkpoint, 'gradient_checkpointing_enable'):
                     try:
                         model_to_checkpoint.gradient_checkpointing_enable()
                         print("Activation checkpointing enabled for the language model.")
                     except Exception as e:
                         print(f"Warning: Failed to enable activation checkpointing: {e}")
                 else:
                     print("Warning: Language model does not have 'gradient_checkpointing_enable' method. Cannot enable activation checkpointing automatically.")
            else:
                 print("Warning: Activation checkpointing enabled in config, but language model is not loaded.")
        else:
             print("Activation checkpointing is disabled in the configuration.")

    def get_input_embeddings(self) -> nn.Embedding:
        """Helper to get the LLM's input embedding layer."""
        if self.language_model is None:
            raise ValueError("Language model not loaded.")
        return self.language_model.get_input_embeddings()

    def encode_image(self, pixel_values: torch.Tensor) -> torch.Tensor | None:
        """Encodes images using the vision tower and extracts features from the specified layer.

        Args:
            pixel_values: Tensor of shape (batch_size, C, H, W).

        Returns:
            Tensor containing the selected image features (batch_size, num_patches, hidden_dim),
            or None if the vision tower failed to load.
        """
        if self.vision_tower is None:
            print("Error: Vision tower not loaded, cannot encode image.")
            return None

        try:
            # Ensure pixel_values are on the same device as the vision tower
            device = self.vision_tower.device
            pixel_values = pixel_values.to(device, dtype=self.vision_tower.dtype)

            # Pass image through vision tower. Request hidden states to access intermediate layers.
            vision_outputs: BaseModelOutputWithPooling = self.vision_tower(
                pixel_values,
                output_hidden_states=True
            )

            # Extract features from the specified layer.
            # hidden_states is a tuple: (embedding_layer_output, layer1_output, ..., last_layer_output)
            # We use vision_feature_layer (e.g., -2 for second-to-last) to index into this tuple.
            image_features = vision_outputs.hidden_states[self.vision_feature_layer]

            # The output includes features for the [CLS] token and patch tokens.
            # LLaVA uses only the patch features for projection.
            # Shape: (batch_size, num_tokens, hidden_dim), where num_tokens = num_patches + 1 (for CLS)
            # We remove the first token ([CLS]) feature.
            image_features = image_features[:, 1:, :] # Shape: (batch_size, num_patches, hidden_dim)

            return image_features

        except Exception as e:
            print(f"Error during image encoding: {e}")
            import traceback
            traceback.print_exc()
            return None

    # --- Step 2.5: Implement Baseline Forward Pass ---
    def forward(self,
                pixel_values: torch.Tensor,
                input_ids: torch.Tensor,
                attention_mask: Optional[torch.Tensor] = None, # Made optional for HF compatibility
                labels: Optional[torch.Tensor] = None # Made optional
               ) -> CausalLMOutputWithPast:
        """Defines the forward pass of the LLaVA model.

        Processes the image, projects features, combines with text embeddings,
        and passes the result through the language model.

        Args:
            pixel_values: Tensor of shape (batch_size, C, H, W).
            input_ids: Tensor of shape (batch_size, sequence_length) containing token IDs
                       and IMAGE_TOKEN_INDEX_PLACEHOLDER markers (-200).
            attention_mask: Optional tensor of shape (batch_size, sequence_length).
                            If None, it will be constructed based on input_ids padding.
            labels: Optional tensor of shape (batch_size, sequence_length) corresponding
                    to input_ids (with -100 masking) for loss calculation.

        Returns:
            Output dictionary from the language model (transformers.CausalLMOutputWithPast),
            typically including 'logits'. If labels are provided, the output includes 'loss'.
        """
        if self.language_model is None or self.vision_tower is None or self.projector is None:
            raise RuntimeError("Model components (LLM, Vision Tower, Projector) are not fully loaded.")

        # 1. Encode Image & Project Features
        image_features = self.encode_image(pixel_values) # (B, P, D_clip)
        if image_features is None:
            raise RuntimeError("Image encoding failed.")
        projected_image_features = self.projector(image_features) # (B, P, D_llm)
        num_image_patches = projected_image_features.shape[1]

        # 2. Prepare inputs for embedding replacement
        # We need input embeddings, not input_ids directly for replacement
        input_ids_clone = input_ids.clone() # Clone to avoid modifying original
        input_ids_clone[input_ids_clone == self.image_token_index_marker] = 0 # Replace marker temporarily (e.g., with pad token ID) for embedding lookup

        # Get base text embeddings
        text_embeddings = self.get_input_embeddings()(input_ids_clone) # (B, S, D_llm)

        # 3. Combine Text and Image Embeddings
        new_input_embeds = []
        new_labels = [] if labels is not None else None
        new_attention_mask = []

        for batch_idx in range(input_ids.shape[0]):
            # Find the placeholder token index in the original input_ids
            image_token_indices = torch.where(input_ids[batch_idx] == self.image_token_index_marker)[0]
            if len(image_token_indices) == 0:
                # Should not happen if data processing is correct
                warnings.warn(f"Image token placeholder {self.image_token_index_marker} not found in batch index {batch_idx}. Skipping image features.")
                # Fallback: use original text embeddings and mask
                new_input_embeds.append(text_embeddings[batch_idx])
                # Construct attention mask if not provided initially
                current_attention_mask = attention_mask[batch_idx] if attention_mask is not None else (input_ids[batch_idx] != tokenizer.pad_token_id).long()
                new_attention_mask.append(current_attention_mask)
                if new_labels is not None and labels is not None:
                    new_labels.append(labels[batch_idx])
                continue

            image_token_start_index = image_token_indices[0].item()

            # Slice text embeddings
            text_emb_before = text_embeddings[batch_idx, :image_token_start_index]
            text_emb_after = text_embeddings[batch_idx, image_token_start_index + 1:]

            # Concatenate to form the final sequence embedding
            cur_new_embed = torch.cat([
                text_emb_before,
                projected_image_features[batch_idx].to(text_embeddings.device, dtype=text_embeddings.dtype), # Ensure device/dtype match
                text_emb_after
            ], dim=0)
            new_input_embeds.append(cur_new_embed)

            # Construct the new attention mask for this sample
            # If original mask was provided, use it; otherwise create from input_ids
            current_attention_mask = attention_mask[batch_idx] if attention_mask is not None else (input_ids[batch_idx] != tokenizer.pad_token_id).long()
            mask_before = current_attention_mask[:image_token_start_index]
            mask_image = torch.ones(num_image_patches, dtype=torch.long, device=current_attention_mask.device)
            mask_after = current_attention_mask[image_token_start_index + 1:]
            cur_new_mask = torch.cat([
                mask_before,
                mask_image,
                mask_after
            ], dim=0)
            new_attention_mask.append(cur_new_mask)

            # Construct the new labels for this sample if needed
            if new_labels is not None and labels is not None:
                label_before = labels[batch_idx, :image_token_start_index]
                label_image = torch.full((num_image_patches,), self.ignore_index, dtype=torch.long, device=labels.device)
                label_after = labels[batch_idx, image_token_start_index + 1:]
                cur_new_label = torch.cat([
                    label_before,
                    label_image,
                    label_after
                ], dim=0)
                new_labels.append(cur_new_label)

        # 4. Pad the sequences in the batch to the maximum length
        # batch_first=True ensures output shape is (B, S_final, D_llm)
        padded_input_embeds = pad_sequence(new_input_embeds, batch_first=True, padding_value=0.0)
        padded_attention_mask = pad_sequence(new_attention_mask, batch_first=True, padding_value=0)
        padded_labels = None
        if new_labels is not None:
            padded_labels = pad_sequence(new_labels, batch_first=True, padding_value=self.ignore_index)

        # 5. Pass combined embeddings and new mask to the language model
        outputs: CausalLMOutputWithPast = self.language_model(
            inputs_embeds=padded_input_embeds,
            attention_mask=padded_attention_mask,
            labels=padded_labels,
            return_dict=True
        )

        return outputs

In [ ]:
show_doc(BaselineLLaVAModel, name='BaselineLLaVAModel')

---

### BaselineLLaVAModel

>      BaselineLLaVAModel (config:dict)

*Baseline LLaVA 1.5 model combining Vision Encoder, Projector, and LLM.

    Loads the specified Vision Encoder (CLIP) and LLM (Vicuna) from Hugging Face Hub,
    along with the Projector. Handles freezing components, LLM embedding resizing,
    and optionally applies LoRA to the LLM based on the configuration.
    Implements `encode_image`, activation checkpointing, and the main `forward` pass.*

In [ ]:
#| test
# Test BaselineLLaVAModel instantiation including LLM loading and embedding resize
import torch, gc # Add torch and gc
from transformers import AutoModelForCausalLM, CLIPVisionModel # Add imports

try:
    config_path = 'configs/config.yaml'
    config = load_config(config_path)
    print(f"Loaded config from {config_path}")
    
    # --- Test with LoRA enabled (assuming config enables it) --- 
    config['model']['peft']['use_lora'] = True # Ensure LoRA is tested
    config['model']['use_activation_checkpointing'] = False # Disable checkpointing for this basic test
    
    baseline_model = BaselineLLaVAModel(config)

    # Basic checks
    assert isinstance(baseline_model, nn.Module)
    assert baseline_model.projector is not None
    assert baseline_model.vision_tower is not None
    assert isinstance(baseline_model.vision_tower, CLIPVisionModel)
    assert baseline_model.language_model is not None # Should be loaded now
    # Check if LLM is wrapped by PeftModel if LoRA is enabled
    if _peft_available and config['model']['peft']['use_lora']:
         assert isinstance(baseline_model.language_model, PeftModel), f"Expected PeftModel, but got {type(baseline_model.language_model)}"
         print("Language model correctly wrapped by PeftModel.")
    else:
         assert isinstance(baseline_model.language_model, AutoModelForCausalLM)
         print("Language model is standard AutoModelForCausalLM (LoRA disabled or peft not available).")
         
    print("BaselineLLaVAModel test passed!")
    print(f"  - Projector exists: {baseline_model.projector is not None} (Type: {type(baseline_model.projector)})")
    print(f"  - Vision Tower exists: {baseline_model.vision_tower is not None} (Type: {type(baseline_model.vision_tower)})")
    print(f"  - Language Model exists: {baseline_model.language_model is not None} (Type: {type(baseline_model.language_model)})")

    # Test encode_image helper (already tested, but good to re-check)
    print("\nTesting encode_image...")
    batch_size = 2
    dummy_pixel_values = torch.randn(batch_size, 3, 336, 336) # Example image batch
    image_features = baseline_model.encode_image(dummy_pixel_values)

    assert image_features is not None
    expected_shape = (batch_size, 576, 1024)
    assert image_features.shape == expected_shape, f"Expected shape {expected_shape}, but got {image_features.shape}"
    print(f"Image features shape: {image_features.shape}")
    print("encode_image test passed!")

    # Test embedding size
    print("\nTesting LLM embedding size...")
    llm_embeddings = baseline_model.get_input_embeddings()
    llm_vocab_size = llm_embeddings.weight.size(0)
    tokenizer_vocab_size = len(tokenizer)
    print(f"LLM embedding vocab size: {llm_vocab_size}")
    print(f"Tokenizer vocab size: {tokenizer_vocab_size}")
    assert llm_vocab_size == tokenizer_vocab_size, \
        f"Mismatch! LLM embedding size ({llm_vocab_size}) != Tokenizer size ({tokenizer_vocab_size})"
    print("LLM embedding size test passed!")

except FileNotFoundError:
    print(f"Config file {config_path} not found. Skipping BaselineLLaVAModel test.")
except ImportError as e:
    print(f"Skipping test due to ImportError: {e}. (Likely `peft` is missing)")
except Exception as e:
    print(f"An error occurred during BaselineLLaVAModel test: {e}")
    import traceback
    traceback.print_exc()

# Clean up large model from memory if running tests in a notebook
# import gc # Already imported
if 'baseline_model' in locals():
    # Ensure model is moved to CPU before deleting if it was on GPU
    if hasattr(baseline_model, 'vision_tower') and baseline_model.vision_tower is not None:
        baseline_model.vision_tower.to('cpu')
    if hasattr(baseline_model, 'language_model') and baseline_model.language_model is not None:
        baseline_model.language_model.to('cpu')
    if hasattr(baseline_model, 'projector') and baseline_model.projector is not None:
        baseline_model.projector.to('cpu')
    del baseline_model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("Cleaned up baseline_model")

Loaded config from configs/config.yaml
Initializing Projector: Input Dim=1024, Output Dim=4096
Loading Vision Tower: openai/clip-vit-large-patch14-336...


Some weights of the model checkpoint at openai/clip-vit-large-patch14-336 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.0.layer_norm1.bias', 'text_model.encoder.layers.0.layer_norm1.weight', 'text_model.encoder.layers.0.layer_norm2.bias', 'text_model.encoder.layers.0.layer_norm2.weight', 'text_model.encoder.layers.0.mlp.fc1.bias', 'text_model.encoder.layers.0.mlp.fc1.weight', 'text_model.encoder.layers.0.mlp.fc2.bias', 'text_model.encoder.layers.0.mlp.fc2.weight', 'text_model.encoder.layers.0.self_attn.k_proj.bias', 'text_model.encoder.layers.0.self_attn.k_proj.weight', 'text_model.encoder.layers.0.self_attn.out_proj.bias', 'text_model.encoder.layers.0.self_attn.out_proj.weight', 'text_model.encoder.layers.0.self_attn.q_proj.bias', 'text_model.encoder.layers.0.self_attn.q_proj.weight', 'text_model.encoder.layers.0.self_attn.v_proj.bias', 'text_model.encoder.layers.0.self_attn.v_proj.weight', 'text_model.encoder.layers.1.layer_norm1.bias', 'text_model.enco

Vision Tower loaded successfully.
Vision Tower weights frozen.
Loading Language Model: lmsys/vicuna-7b-v1.5...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Language Model loaded successfully.
Base Language Model weights frozen.
Applying LoRA...
LoRA applied. Config: r=8, alpha=16, dropout=0.05, modules=['q_proj', 'v_proj']
trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199
LLM embedding size already matches tokenizer size. No resizing needed.
Activation checkpointing is disabled in the configuration.
BaselineLLaVAModel test passed!
  - Projector exists: True (Type: <class '__main__.LLaVAProjector'>)
  - Vision Tower exists: True (Type: <class 'transformers.models.clip.modeling_clip.CLIPVisionModel'>)
  - Language Model exists: True (Type: <class 'peft.peft_model.PeftModelForCausalLM'>)

Testing encode_image...
Image features shape: torch.Size([2, 576, 1024])
encode_image test passed!

Testing LLM embedding size...
LLM embedding vocab size: 32001
Tokenizer vocab size: 32001
LLM embedding size test passed!
Cleaned up baseline_model


---

### Forward Pass Test

In [ ]:
#| test
# Test the implemented forward pass
import torch, gc
from transformers.modeling_outputs import CausalLMOutputWithPast

try:
    config_path = 'configs/config.yaml'
    config = load_config(config_path)
    print(f"Loaded config from {config_path}")
    
    # Ensure LoRA is enabled for this test if peft is available
    if _peft_available:
         config['model']['peft']['use_lora'] = True
    else:
         config['model']['peft']['use_lora'] = False
    config['model']['use_activation_checkpointing'] = False # Ensure checkpointing is off for this basic test

    # Instantiate the model
    # Reduce memory usage for testing if possible (e.g., use smaller test models if available)
    # Or ensure enough memory is allocated
    forward_test_model = BaselineLLaVAModel(config)

    # Set model to evaluation mode for testing (disables dropout etc.)
    forward_test_model.eval()

    print("Running forward pass test...")
    # Prepare dummy inputs
    batch_size = 2
    seq_len = 15 # Short sequence for testing
    img_size = 336 # From config
    num_patches = 576 # (336/14)^2
    llm_hidden_dim = config['model']['projector']['output_dim']
    tokenizer_vocab_size = len(tokenizer)

    dummy_pixel_values = torch.randn(batch_size, 3, img_size, img_size)
    # Create input_ids with placeholder
    dummy_input_ids = torch.randint(1, tokenizer_vocab_size, (batch_size, seq_len), dtype=torch.long)
    placeholder_idx = 5 # Place the image token marker at index 5
    dummy_input_ids[:, placeholder_idx] = IMAGE_TOKEN_INDEX_PLACEHOLDER
    # Create attention mask (mask padding if any, here assume no padding initially)
    dummy_attention_mask = torch.ones_like(dummy_input_ids)
    # Create labels (copy input_ids, mask placeholder and potentially prompt)
    dummy_labels = dummy_input_ids.clone()
    dummy_labels[:, :placeholder_idx+1] = IGNORE_INDEX # Mask prompt + image token

    print("Created dummy inputs.")
    print(f"Original input_ids shape: {dummy_input_ids.shape}")

    # Perform forward pass
    with torch.no_grad(): # Disable gradient calculation for testing inference
         outputs = forward_test_model(
            pixel_values=dummy_pixel_values,
            input_ids=dummy_input_ids,
            attention_mask=dummy_attention_mask,
            labels=dummy_labels
         )

    # Check output type and attributes
    assert isinstance(outputs, CausalLMOutputWithPast)
    assert hasattr(outputs, 'logits')
    assert outputs.logits is not None
    assert hasattr(outputs, 'loss') # Should have loss since labels were provided
    assert outputs.loss is not None

    # Check output shapes
    expected_seq_len = seq_len - 1 + num_patches
    expected_logits_shape = (batch_size, expected_seq_len, tokenizer_vocab_size)

    # Print shapes from inside forward pass (for debugging if needed)
    # Accessing internal shapes requires modification or inspection
    # Instead, check the final output shapes
    print(f"Padded inputs_embeds shape: ({batch_size}, {expected_seq_len}, {llm_hidden_dim})") # Expected shape after padding
    print(f"Padded attention_mask shape: ({batch_size}, {expected_seq_len})") # Expected shape after padding
    print(f"Padded labels shape: ({batch_size}, {expected_seq_len})") # Expected shape after padding

    assert outputs.logits.shape == expected_logits_shape, \
        f"Expected logits shape {expected_logits_shape}, but got {outputs.logits.shape}"
    print(f"Output logits shape: {outputs.logits.shape}")
    print(f"Output loss: {outputs.loss}")
    print("Forward pass test successful!")

except FileNotFoundError:
    print(f"Config file {config_path} not found. Skipping forward pass test.")
except ImportError as e:
     print(f"Skipping forward pass test due to ImportError: {e}. (Likely `peft` is missing)")
except Exception as e:
    print(f"An error occurred during forward pass test: {e}")
    import traceback
    traceback.print_exc()

# Clean up
if 'forward_test_model' in locals():
    if hasattr(forward_test_model, 'vision_tower') and forward_test_model.vision_tower is not None:
        forward_test_model.vision_tower.to('cpu')
    if hasattr(forward_test_model, 'language_model') and forward_test_model.language_model is not None:
        forward_test_model.language_model.to('cpu')
    if hasattr(forward_test_model, 'projector') and forward_test_model.projector is not None:
        forward_test_model.projector.to('cpu')
    del forward_test_model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("Cleaned up forward_test_model")


Loaded config from configs/config.yaml
Initializing Projector: Input Dim=1024, Output Dim=4096
Loading Vision Tower: openai/clip-vit-large-patch14-336...


Some weights of the model checkpoint at openai/clip-vit-large-patch14-336 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.0.layer_norm1.bias', 'text_model.encoder.layers.0.layer_norm1.weight', 'text_model.encoder.layers.0.layer_norm2.bias', 'text_model.encoder.layers.0.layer_norm2.weight', 'text_model.encoder.layers.0.mlp.fc1.bias', 'text_model.encoder.layers.0.mlp.fc1.weight', 'text_model.encoder.layers.0.mlp.fc2.bias', 'text_model.encoder.layers.0.mlp.fc2.weight', 'text_model.encoder.layers.0.self_attn.k_proj.bias', 'text_model.encoder.layers.0.self_attn.k_proj.weight', 'text_model.encoder.layers.0.self_attn.out_proj.bias', 'text_model.encoder.layers.0.self_attn.out_proj.weight', 'text_model.encoder.layers.0.self_attn.q_proj.bias', 'text_model.encoder.layers.0.self_attn.q_proj.weight', 'text_model.encoder.layers.0.self_attn.v_proj.bias', 'text_model.encoder.layers.0.self_attn.v_proj.weight', 'text_model.encoder.layers.1.layer_norm1.bias', 'text_model.enco

Vision Tower loaded successfully.
Vision Tower weights frozen.
Loading Language Model: lmsys/vicuna-7b-v1.5...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Language Model loaded successfully.
Base Language Model weights frozen.
Applying LoRA...
LoRA applied. Config: r=8, alpha=16, dropout=0.05, modules=['q_proj', 'v_proj']
trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199
LLM embedding size already matches tokenizer size. No resizing needed.
Activation checkpointing is disabled in the configuration.
Running forward pass test...
Created dummy inputs.
Original input_ids shape: torch.Size([2, 15])
Padded inputs_embeds shape: (2, 586, 4096)
Padded attention_mask shape: (2, 586)
Padded labels shape: (2, 586)
Output logits shape: torch.Size([2, 586, 32001])
Output loss: tensor(10.3799, grad_fn=<NllLossBackward0>)
Forward pass test successful!
Cleaned up forward_test_model


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()